In [1]:
import os
import cv2
import numpy as np
from typing import Generator
import pandas as pd

#### Constant variables

In [2]:
TRANSFORMED_DATA_103_PATH = "C:/Users/diogo/Desktop/Tese/Resultados/transformed_coords_103.csv"
VIDEOS_103_DIR_PATH = "C:/Users/diogo/Desktop/Tese/Dados/Videos/14.03.2022/20220314_1.03_blurred"
SOURCE_VIDEO_PATH = "C:/Users/diogo/Desktop/Tese/Dados/Videos/14.03.2022/20220314_1.03_blurred/20220314_1.03_1_11_blurred.mp4"
SAVE_IMG_PATH = "C:/Users/diogo/Desktop/Tese/Resultados/background.jpg"

#### Load video and dataframe

In [3]:
# load and sort excel data by Customer_id
def load_and_sort_df(path):
    df = pd.read_csv(path)
    return df.sort_values(["Customer_id", "new_frame_id"], ascending=True, ignore_index=True)

# get video capture
def get_video_capture(video_path: str) -> cv2.VideoCapture:
    return cv2.VideoCapture(video_path)

# generate frame list from a video capture
def generate_frames(video_capture: cv2.VideoCapture) -> Generator[np.ndarray, None, None]:
    while video_capture.isOpened():
        success, frame = video_capture.read()
        if not success:
            break
        yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# get image of the store to set as a background
def get_background():
    video_name = f"20220314_1.03_1_1_blurred.mp4"
    video_capture = get_video_capture(f"{VIDEOS_103_DIR_PATH}/{video_name}")
    frame_list = list(generate_frames(video_capture))
    return frame_list[0]

#### Operations

In [4]:
# draw rectangles in store background
def draw_bboxes(df, customer):
    customer_data = df.loc[df.Customer_id == customer].values.tolist()
    image = get_background()

    counter = 0
    for row in customer_data:
        start_point = (int(row[9]), int(row[10]))
        end_point = (int(row[11]), int(row[12]))
        color = (255, counter, 0)
        image = cv2.rectangle(image, start_point, end_point, color, thickness=2)
        counter += 10

    return image
    #cv2.imshow("image drawn", image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

#### GUI

In [5]:
from tkinter import *
from PIL import ImageTk, Image

def run_app(df, customer):
    # Create the main window
    window = Tk()

    # Set the window title
    window.title("Anotation Helper")

    # Set the window size
    window.geometry("1920x1080")
    window.state('zoomed')

    # Load the image file
    #img = Image.open("background.jpg")
    img = draw_bboxes(df, customer)

    blue,green,red = cv2.split(img)
    img = cv2.merge((red,green,blue))
    im = Image.fromarray(img)
    im = im.resize((1440, 810))
    imgtk = ImageTk.PhotoImage(image=im)

    # Create a label to display the image
    image_label = Label(window, image=imgtk)
    image_label.pack()

    # Create text widget
    T = Text(window, height = 5, width = 52)
    customer_text = f"Customer number: {customer}"
    T.insert(END, customer_text)
    T.pack()

    #Define a function to close the window
    def close_window():
        window.destroy()
    button = Button(window, text="CLOSE", command=close_window)
    button.pack()

    # Start the main loop to display the GUI
    window.mainloop()

In [6]:
df = load_and_sort_df(TRANSFORMED_DATA_103_PATH)
for customer in df["Customer_id"].unique():
    run_app(df, customer)

KeyboardInterrupt: 

: 